In [1]:
import numpy as np
import pandas as pd
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import catboost as cat
from catboost import CatBoostClassifier
import xgboost as xgb

## Загрузим данные

In [2]:
%%time
edges = pd.read_csv('./edges.csv')
ids = pd.read_csv('./ids.csv')
vertices = pd.read_csv('./vertices.csv')

Wall time: 2.84 s


In [3]:
vertices['main_okved'] = vertices['main_okved'].astype(str)

In [4]:
np.random.seed(7777)

# Визуализация

## Обучим модель

In [5]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [33]:
# для каждой вершины из ids с помощью catboost найдем 1000 самых вероятных ребер
for i in tqdm.tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1
    
    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
    
    
    X = df[['main_okved', 'region_code', 'company_type']]
    y = df['target']
    
    model = CatBoostClassifier(iterations=600, verbose=False)
    cat_features = [0,1,2] # все признаки категориальные
    
    model.fit(X, y, cat_features)

    preds = model.predict_proba(X)[:,1]

    df['preds'] = preds
    df['id_2'] = i
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2']
    
    result = result.append(res, ignore_index=True, sort=False)



  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  1%|▊                                                                              | 1/100 [05:24<8:55:06, 324.31s/it]

  2%|█▌                                                                             | 2/100 [10:52<8:51:32, 325.43s/it]

  3%|██▎                                                                            | 3/100 [16:24<8:49:34, 327.57s/it]

  4%|███▏                                                                           | 4/100 [21:54<8:45:09, 328.22s/it]

  5%|███▉                                                                           | 5/100 [27:09<8:33:17, 324.19s/it]

  6%|████▋                                                                          | 6/100 [32:18<8:21:00, 319.79s/it]

  7%|█████▌                                                                         | 7/100 [37:24<8:09:00, 315.49s/it]

  8%|██████▎                  

## Результат готов к отправке

In [13]:
result.to_csv('submission4000.csv', index = False)

In [39]:
res

,id_1,id_2
0,537164,1244877
1,1390385,1244877
2,99199,1244877
3,591239,1244877
4,483817,1244877
...,...,...
995,287342,1244877
996,217663,1244877
997,703343,1244877
998,287365,1244877


# Второй вариант обучения

In [11]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [12]:

# для каждой вершины из ids с помощью catboost найдем 1000 самых вероятных ребер
for i in tqdm.tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1
    
    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
    
    
    X = df[['main_okved', 'region_code', 'company_type']]
    y = df['target']
    
    model = CatBoostClassifier(iterations=300, verbose=False,loss_function='MultiClass',learning_rate=0.03, eval_metric='Accuracy'
)
    cat_features = [0,1,2] # все признаки категориальные
    
    model.fit(X, y, cat_features)

    preds = model.predict_proba(X)[:,1]

    df['preds'] = preds
    df['id_2'] = i
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2']
    
    result = result.append(res, ignore_index=True, sort=False)

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [4:29:05<00:00, 161.45s/it]


In [10]:
result

,id_1,id_2
0,188225,524354
1,1195897,524354
2,1447932,524354
3,1434000,524354
4,1295314,524354
...,...,...
179995,165027,1244877
179996,957332,1244877
179997,982526,1244877
179998,340093,1244877


In [13]:
result

,id_1,id_2
0,188225,524354
1,1195897,524354
2,1447932,524354
3,1434000,524354
4,1295314,524354
...,...,...
99995,1493161,1244877
99996,1244903,1244877
99997,13308,1244877
99998,139347,1244877
